In [27]:
import pandas as pd

In [28]:
#数据分割
df = pd.read_csv("train.csv")
X = df.iloc[:,1:-1] #直接去掉Id列了
y = df.iloc[:,-1]

df_new = pd.read_csv("test.csv")
X_new = df_new.iloc[:,1:]

#X.drop(columns=['PoolQC','MiscFeature','Alley','Fence'],inplace=True)
#X_new.drop(columns=['PoolQC','MiscFeature','Alley','Fence'],inplace=True)

In [29]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression,RidgeCV,Lasso,SGDRegressor
from sklearn.model_selection import cross_val_score

In [30]:
#set up preprocessing for numeric columns
imp_median = SimpleImputer(strategy='mean', add_indicator=True)
scaler = StandardScaler()

In [31]:
#set up preprocessing for categorical columns
imp_constant = SimpleImputer(strategy='constant') #用constant字符填充缺失值
ohe = OneHotEncoder(handle_unknown='ignore') #必须要设定这个参数！原因可见ML_preprocess_with_pandas_ver2.0中In[11]上面的解释
#ore = OrdinalEncoder(handle_unknown='use_encoded_value')

In [32]:
#select columns by data type
num_cols = make_column_selector(dtype_include='number')
cat_cols = make_column_selector(dtype_exclude='number')

In [33]:
#do all preprocessing
preprocessor = make_column_transformer(
    (make_pipeline(imp_median, scaler), num_cols),
    (make_pipeline(imp_constant, ohe), cat_cols))

In [37]:
from sklearn.ensemble import VotingRegressor,RandomForestRegressor
#create a pipeline
sgd = SGDRegressor()
rfr = RandomForestRegressor()

vr = VotingRegressor([('sgd',sgd),('rfr',rfr)],weights=(1,2))

pipe = make_pipeline(preprocessor, vr)

In [20]:
pipe.steps

[('columntransformer',
  ColumnTransformer(transformers=[('pipeline-1',
                                   Pipeline(steps=[('simpleimputer',
                                                    SimpleImputer(add_indicator=True)),
                                                   ('standardscaler',
                                                    StandardScaler())]),
                                   <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7704BE820>),
                                  ('pipeline-2',
                                   Pipeline(steps=[('simpleimputer',
                                                    SimpleImputer(strategy='constant')),
                                                   ('onehotencoder',
                                                    OneHotEncoder(handle_unknown='ignore'))]),
                                   <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7704BE2E0>)])),
 ('v

In [21]:
# from sklearn.model_selection import GridSearchCV

# param_grid = {}
# param_grid['votingregressor__rfr__n_estimators'] = [100,500]
# param_grid['votingregressor__sgd__alpha'] = [0.001,0.01]

In [22]:
# grid = GridSearchCV(pipe, param_grid, cv=5, scoring='r2')

In [23]:
# %time grid.fit(X,y)

Wall time: 13min 12s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(add_indicator=True)),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7704BE820>),
                                                                        ('pipeline-2',
                                                                         Pipeline(steps=[('simpleimputer',...
                     

In [38]:
#cross_validate the pipeline
print(cross_val_score(pipe, X, y).mean())

0.8682542534559765


In [24]:
# print(grid.best_params_)
# print(grid.best_score_)

{'votingregressor__rfr__n_estimators': 500, 'votingregressor__sgd__alpha': 0.001}
0.8707012450093143


In [25]:
# a = grid.predict(X_new)

In [39]:
# #fit the pipeline and make predictions
pipe.fit(X, y)
a = pipe.predict(X_new)
print(a)

[116839.20364831 157827.96334482 176827.06988431 ... 155213.83667774
 113022.54693407 231213.63759802]


In [40]:
pd.DataFrame({'id':df_new.Id,'SalePrice':a}).set_index('id').to_csv('sub_houseprice_ver2.0_rft_sgd_without_ore.csv')